In [ ]:
# To run the folloing codes, you need to install the following packages
# Tensorflow-gpu = 2.3.0
# tensorRT = 6.0.1 
# CUDA = 10.2
# !pip install Pillow
# Instructions: Get TRT FP16 model from the ./fps/TRT_weights directory. Get the sample images from ./fps/Sample_images directory.
# CHange the path of image and TRT model directory in the following code. 

In [1]:
### The following code meaure the Throughout (Frame Per Second (FPS))
### of ENet tensorRT FP 16 model at 1024 x 2048 input resolution (Cityscapes dataset).

from tensorflow import keras
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

batch_size = 4
h =1024
w =2048
batched_input = np.zeros((batch_size, h, w, 3), dtype=np.float32)

for i in range(batch_size):
  ### path of the image directory  
  img_path = '/home/jovyan/work/Keras_tensorRT_models/data/img%d.png' % (i % 4)
  img = image.load_img(img_path, target_size=(h, w))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  #x = preprocess_input(x)
  batched_input[i, :] = x
batched_input = tf.constant(batched_input)
print('batched_input shape: ', batched_input.shape)

def benchmark_tftrt(input_saved_model):
    saved_model_loaded = tf.saved_model.load(input_saved_model, tags=[tag_constants.SERVING])
    infer = saved_model_loaded.signatures['serving_default']

    N_warmup_run = 20
    N_run = 200
    elapsed_time = []

    for i in range(N_warmup_run):
      labeling = infer(batched_input)

    for i in range(N_run):
      start_time = time.time()
      labeling = infer(batched_input)
      #prob = labeling['probs'].numpy()
      end_time = time.time()
      elapsed_time = np.append(elapsed_time, end_time - start_time)
      #if i % 20 == 0:
        #print('Step {}: {:4.1f}ms'.format(i, (elapsed_time[-50:].mean()) * 1000))

    print('Throughput: {:.0f} images/s'.format(N_run * batch_size / elapsed_time.sum()))

### load the TRT FP16 model    
FPS16 = benchmark_tftrt('ENet_cityscapes_TFTRT_FP16')

2022-10-29 14:28:22.573347: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-29 14:28:24.376609: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-10-29 14:28:24.550890: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties:
pciBusID: 0000:1b:00.0 name: NVIDIA TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-10-29 14:28:24.550963: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-29 14:28:24.553531: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-10-29 14:28:24.556159: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10

In [1]:
### The following code meaure the Throughout (Frame Per Second (FPS))
### of ENet tensorRT FP 16 model at 384 x 1280 input resolution (KITTI dataset).

from tensorflow import keras
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

batch_size = 4
h =384
w =1280
batched_input = np.zeros((batch_size, h, w, 3), dtype=np.float32)

for i in range(batch_size):
  ### path of the image directory  
  img_path = '/home/jovyan/work/Keras_tensorRT_models/data/img%d.png' % (i % 4)
  img = image.load_img(img_path, target_size=(h, w))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  #x = preprocess_input(x)
  batched_input[i, :] = x
batched_input = tf.constant(batched_input)
print('batched_input shape: ', batched_input.shape)

def benchmark_tftrt(input_saved_model):
    saved_model_loaded = tf.saved_model.load(input_saved_model, tags=[tag_constants.SERVING])
    infer = saved_model_loaded.signatures['serving_default']

    N_warmup_run = 20
    N_run = 200
    elapsed_time = []

    for i in range(N_warmup_run):
      labeling = infer(batched_input)

    for i in range(N_run):
      start_time = time.time()
      labeling = infer(batched_input)
      #prob = labeling['probs'].numpy()
      end_time = time.time()
      elapsed_time = np.append(elapsed_time, end_time - start_time)
      #if i % 20 == 0:
        #print('Step {}: {:4.1f}ms'.format(i, (elapsed_time[-50:].mean()) * 1000))

    print('Throughput: {:.0f} images/s'.format(N_run * batch_size / elapsed_time.sum()))

### load the TRT FP16 model        
FPS16 = benchmark_tftrt('ENet_kitti_TFTRT_FP16')

2022-10-29 13:28:30.401113: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-29 13:28:32.017665: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-10-29 13:28:32.193747: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties:
pciBusID: 0000:1b:00.0 name: NVIDIA TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-10-29 13:28:32.193832: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-29 13:28:32.196536: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-10-29 13:28:32.199243: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10

In [1]:
### The following code meaure the Throughout (Frame Per Second (FPS))
### of ENet tensorRT FP 16 model at 640 x 896 input resolution (CamVid dataset).

from tensorflow import keras
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

batch_size = 4
h =640
w =896
batched_input = np.zeros((batch_size, h, w, 3), dtype=np.float32)

for i in range(batch_size):
  ### path of the image directory  
  img_path = '/home/jovyan/work/Keras_tensorRT_models/data/img%d.png' % (i % 4)
  img = image.load_img(img_path, target_size=(h, w))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  #x = preprocess_input(x)
  batched_input[i, :] = x
batched_input = tf.constant(batched_input)
print('batched_input shape: ', batched_input.shape)

def benchmark_tftrt(input_saved_model):
    saved_model_loaded = tf.saved_model.load(input_saved_model, tags=[tag_constants.SERVING])
    infer = saved_model_loaded.signatures['serving_default']

    N_warmup_run = 20
    N_run = 200
    elapsed_time = []

    for i in range(N_warmup_run):
      labeling = infer(batched_input)

    for i in range(N_run):
      start_time = time.time()
      labeling = infer(batched_input)
      #prob = labeling['probs'].numpy()
      end_time = time.time()
      elapsed_time = np.append(elapsed_time, end_time - start_time)
      #if i % 20 == 0:
        #print('Step {}: {:4.1f}ms'.format(i, (elapsed_time[-50:].mean()) * 1000))

    print('Throughput: {:.0f} images/s'.format(N_run * batch_size / elapsed_time.sum()))

### load the TRT FP16 model    
FPS16 = benchmark_tftrt('ENet_camvid_TFTRT_FP16')

2022-10-29 13:32:03.195656: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-29 13:32:04.825983: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-10-29 13:32:05.116313: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties:
pciBusID: 0000:1b:00.0 name: NVIDIA TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-10-29 13:32:05.116406: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-29 13:32:05.119292: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-10-29 13:32:05.123436: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10